In [2]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import requests
import time
from datetime import datetime
# Import the API key.
from config import weather_api_key

In [3]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
# zip packs each data point object that has the same index in each array into a tuple (0, 1)
lat_lngs = zip(lats, lngs)
lat_lngs

In [4]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [5]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

749

In [6]:
# Starting URL for Weather Map API Call.
# units = imperial will give us F not celsius
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [26]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
## initialized to begin at 1 since we want the iterations to start at 1 later
record_count = 1
set_count = 1

# Loop through all the cities in the list.
# enumerate will allow us to pull the city name value based off index position
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        # the one minute sleep after 50 cities will prevent us from exceeding the 60 call per minute free tier limit
        time.sleep(60)

    # Create endpoint URL with each city.
    ## we are removing blank spaces in city names and replacing with url style concats
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # description is stored as a list of objects within weather so weather needs to be separated out first
        weather = city_weather['weather']
        # then we are able to index the new weather object and retrieve description
        city_description = weather[0]['description']
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date,
                          "Current Description":city_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | attawapiskat
City not found. Skipping...
Processing Record 2 of Set 1 | new norfolk
Processing Record 3 of Set 1 | marsh harbour
Processing Record 4 of Set 1 | qaanaaq
Processing Record 5 of Set 1 | barrow
Processing Record 6 of Set 1 | olafsvik
Processing Record 7 of Set 1 | strathmore
Processing Record 8 of Set 1 | la ronge
Processing Record 9 of Set 1 | georgetown
Processing Record 10 of Set 1 | krasnokamensk
Processing Record 11 of Set 1 | ushuaia
Processing Record 12 of Set 1 | tiksi
Processing Record 13 of Set 1 | albany
Processing Record 14 of Set 1 | port elizabeth
Processing Record 15 of Set 1 | punta arenas
Processing Record 16 of Set 1 | port hardy
Processing Record 17 of Set 1 | cidreira
Processing Record 18 of Set 1 | dzhusaly
City not found. Skipping...
Processing Record 19 of Set 1 | butaritari
Processing Record 20 of Set 1 | nikolskoye
Processing Record 21 of Set 1 | samarai
Proce

In [27]:
len(city_data)

691

In [28]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
0,New Norfolk,-42.7826,147.0587,42.93,81,53,2.57,AU,2022-09-03 20:11:37,broken clouds
1,Marsh Harbour,26.5412,-77.0636,84.36,66,100,10.98,BS,2022-09-03 20:14:45,overcast clouds
2,Qaanaaq,77.4840,-69.3632,38.82,96,100,9.24,GL,2022-09-03 20:15:54,light rain
3,Barrow,71.2906,-156.7887,39.22,93,100,8.05,US,2022-09-03 20:15:54,overcast clouds
4,Olafsvik,64.8945,-23.7142,49.42,72,100,6.67,IS,2022-09-03 20:15:54,overcast clouds
5,Strathmore,51.0501,-113.3852,91.87,24,0,9.22,CA,2022-09-03 20:15:08,clear sky
6,La Ronge,55.1001,-105.2842,89.82,19,40,12.66,CA,2022-09-03 20:15:55,scattered clouds
7,Georgetown,5.4112,100.3354,80.53,93,20,2.30,MY,2022-09-03 20:13:27,thunderstorm
8,Krasnokamensk,50.0979,118.0369,56.37,71,100,3.65,RU,2022-09-03 20:15:55,overcast clouds
9,Ushuaia,-54.8000,-68.3000,37.06,93,100,6.91,AR,2022-09-03 20:12:27,light rain


In [30]:
cols = ['City','Country','Date','Lat','Lng','Max Temp','Humidity','Cloudiness','Wind Speed', 'Current Description']

city_data_df = city_data_df[cols]

city_data_df.head()

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,New Norfolk,AU,2022-09-03 20:11:37,-42.7826,147.0587,42.93,81,53,2.57,broken clouds
1,Marsh Harbour,BS,2022-09-03 20:14:45,26.5412,-77.0636,84.36,66,100,10.98,overcast clouds
2,Qaanaaq,GL,2022-09-03 20:15:54,77.4840,-69.3632,38.82,96,100,9.24,light rain
3,Barrow,US,2022-09-03 20:15:54,71.2906,-156.7887,39.22,93,100,8.05,overcast clouds
4,Olafsvik,IS,2022-09-03 20:15:54,64.8945,-23.7142,49.42,72,100,6.67,overcast clouds


In [31]:
# Create the output file (CSV).
output_data_file = "Weather_Database/WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")